In [28]:
from datetime import datetime
import pandas as pd
import requests
import json
import boto3
import sqlalchemy

from requests.auth import HTTPBasicAuth


In [61]:
url = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SESAI/INDICADORES_COVID19/Anexo+B+-+Popula%C3%A7%C3%A3o+e+Trabalhadores+%E2%80%93+Jan_Mar_2022.xlsx"

In [62]:
df = pd.read_csv(url)

MemoryError: 

In [48]:
df = pd.DataFrame(dados['hits'])
df.head()

,_index,_type,_id,_score,_source
0,desc-imunizacao-v5,_doc,630f1d0a-129d-4a55-8f88-b0912de5b97d-i0b0,1.0,{'document_id': '630f1d0a-129d-4a55-8f88-b0912...
1,desc-imunizacao-v5,_doc,a26c716a-c250-4dcf-a4fb-63ca3d804cce-i0b0,1.0,{'document_id': 'a26c716a-c250-4dcf-a4fb-63ca3...
2,desc-imunizacao-v5,_doc,743f4a6c-7acc-4c05-8eed-2ec6c9201468-i0b0,1.0,{'document_id': '743f4a6c-7acc-4c05-8eed-2ec6c...
3,desc-imunizacao-v5,_doc,7c822606-2809-4ac3-81ae-0ba1622c7c17-i0b0,1.0,{'document_id': '7c822606-2809-4ac3-81ae-0ba16...
4,desc-imunizacao-v5,_doc,bca6c946-ad52-4de6-ba06-88786746b828-i0b0,1.0,{'document_id': 'bca6c946-ad52-4de6-ba06-88786...


In [51]:
from pandas.io.json import json_normalize
df = json_normalize(df['_source'])

/tmp/ipykernel_2673/1537383249.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(df['_source'])


In [54]:
df.head()

,document_id,paciente_id,paciente_enumSexoBiologico,vacina_nome,dt_deleted,vacina_codigo,paciente_racaCor_valor,sistema_origem,paciente_endereco_coIbgeMunicipio,paciente_dataNascimento,...,vacina_categoria_nome,vacina_fabricante_referencia,status,vacina_categoria_codigo,vacina_descricao_dose,paciente_idade,vacina_fabricante_nome,vacina_grupoAtendimento_nome,estalecimento_noFantasia,paciente_racaCor_codigo
0,630f1d0a-129d-4a55-8f88-b0912de5b97d-i0b0,291a1210508732aacc8c1e809a7775d52d8c6818996845...,M,COVID-19 PFIZER - COMIRNATY,None,87,AMARELA,VACIVIDA,352310,1959-11-08,...,Faixa Etária,None,final,2,Reforço,62,PFIZER,Pessoas de 18 a 64 anos,UBS JARDIM PAINEIRA,04
1,a26c716a-c250-4dcf-a4fb-63ca3d804cce-i0b0,a39ff527eebf6dc24bde3f52504b39838bb723491b6f37...,M,COVID-19 SINOVAC/BUTANTAN - CORONAVAC,None,86,SEM INFORMACAO,VACIVIDA,354850,1960-11-25,...,Faixa Etária,None,final,2,Reforço,61,SINOVAC/BUTANTAN,Pessoas de 18 a 64 anos,UNIDADE DE SAUDE DA FAMILIA DO BOM RETIRO,99
2,743f4a6c-7acc-4c05-8eed-2ec6c9201468-i0b0,4869df692d06f23cbb274b6e10129bad61279f15760415...,F,COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,None,85,BRANCA,Novo PNI,316470,2002-07-27,...,Faixa Etária,Organization/149,final,2,Reforço,19,ASTRAZENECA/FIOCRUZ,Pessoas de 18 a 64 anos,POSTO DE PUERICULTURA MUNICIPAL,01
3,7c822606-2809-4ac3-81ae-0ba1622c7c17-i0b0,09ad7b73878d3cc0cbf2137850258d00611dfc0f61d4c6...,M,COVID-19 PFIZER - COMIRNATY,None,87,SEM INFORMACAO,VACIVIDA,355030,2004-06-26,...,Faixa Etária,None,final,2,Reforço,17,PFIZER,Pessoas de 12 a 17 anos,UBS VILA ALPINA DR HERMINIO MOREIRA,99
4,bca6c946-ad52-4de6-ba06-88786746b828-i0b0,160dac28e5e42f0a5d8d51cc11d441032a21f4e7e9d7ab...,M,COVID-19 PFIZER - COMIRNATY,None,87,AMARELA,Novo PNI,290460,1994-08-20,...,Faixa Etária,Organization/28290,final,2,Reforço,27,PFIZER,Pessoas de 18 a 64 anos,UNIDADE DE SAUDE DA FAMILIA DR PAULO VARGAS,04


In [108]:
url = "https://apisidra.ibge.gov.br/values/t/6381/n1/all/v/all/p/all/d/m?formato=json"
data = requests.get(url)
data = data.json() 

df = pd.DataFrame(data, header=True)

In [109]:
df.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Brasil (Código),Brasil,Variável (Código),Variável,Trimestre Móvel (Código),Trimestre Móvel
1,1,Brasil,2,%,8.0,1,Brasil,4099,"Taxa de desocupação, na semana de referência, ...",201203,jan-fev-mar 2012
2,1,Brasil,2,%,7.8,1,Brasil,4099,"Taxa de desocupação, na semana de referência, ...",201204,fev-mar-abr 2012
3,1,Brasil,2,%,7.7,1,Brasil,4099,"Taxa de desocupação, na semana de referência, ...",201205,mar-abr-mai 2012
4,1,Brasil,2,%,7.6,1,Brasil,4099,"Taxa de desocupação, na semana de referência, ...",201206,abr-mai-jun 2012
